In [34]:
# import shutil

# # Specify the directory to delete
# dir_path = './results_bert_model_1'

# # Delete the directory and its contents
# shutil.rmtree(dir_path)

# print(f"All data in '{dir_path}' has been deleted.")

All data in './results_bert_model_1' has been deleted.


In [4]:
! pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_syst

In [5]:
!pip install git+https://github.com/huggingface/transformers.git

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-7xzko2so
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-7xzko2so
  Resolved https://github.com/huggingface/transformers.git to commit 4d3b1076a17b72f68c7332008b667c22e81d8f94
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.49.0.dev0-py3-none-any.whl size=10589067 sha256=1aa587c24d16298b0946ca1a66d0937626314e566f4f4e27dfb1db1eca9fdf95
  Stored in directory: /tmp/pip-ephem-wheel-cache-d7iybxpk/wheels/32/4b/78/f195c684dd3a9ed21f3b39fe8f85b48df7918581b6437be143
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.47.1
    Uninstalling transformers-4.47.1:
      Successfully uninstalled transformers-4.47.1


In [6]:
import re
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding, EarlyStoppingCallback
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [35]:
#from datasets import load_dataset

# Load PAWS Subsets
#labeled_final = load_dataset("paws", "labeled_final")

#load MRPC Dataset
labeled_final = load_dataset("glue", "mrpc")

In [8]:
labeled_final

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

In [9]:
labeled_final['train'][0]

{'sentence1': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .',
 'sentence2': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .',
 'label': 1,
 'idx': 0}

In [36]:
labeled_final['train'].features

{'sentence1': Value(dtype='string', id=None),
 'sentence2': Value(dtype='string', id=None),
 'label': ClassLabel(names=['not_equivalent', 'equivalent'], id=None),
 'idx': Value(dtype='int32', id=None)}

In [37]:
# Define text preprocessing function
def preprocess_text(data):
    # Compile regex patterns once
    url_pattern = re.compile(r"http\S+|www\S+|https\S+")
    space_pattern = re.compile(r'\s+')
    contractions = {
        "can't": "cannot", "won't": "will not", "n't": " not", "'re": " are",
        "'s": " is", "'d": " would", "'ll": " will", "'t": " not", "'ve": " have", "'m": " am"
    }

    def clean_sentence(sentence):
        # Remove extra spaces
        sentence = space_pattern.sub(' ', sentence.strip())
        # Remove URLs
        sentence = url_pattern.sub('', sentence)
        # Convert to lowercase
        sentence = sentence.lower()
        # Normalize contractions
        sentence = ' '.join([contractions.get(word, word) for word in sentence.split()])
        return sentence

    # Apply preprocessing to both sentence1 and sentence2
    data['sentence1'] = [clean_sentence(sent) for sent in data['sentence1']]
    data['sentence2'] = [clean_sentence(sent) for sent in data['sentence2']]
    return data

In [38]:
# Apply preprocessing to the dataset
train_dataset = labeled_final["train"].map(preprocess_text, batched=True)
valid_dataset = labeled_final["validation"].map(preprocess_text, batched=True)
test_dataset = labeled_final["test"].map(preprocess_text, batched=True)

In [39]:
train_dataset[0]

{'sentence1': 'amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .',
 'sentence2': 'referring to him as only " the witness " , amrozi accused his brother of deliberately distorting his evidence .',
 'label': 1,
 'idx': 0}

In [40]:
# train = labeled_final['train'].select(range(500))
# test = labeled_final['test'].select(range(50))
# valid = labeled_final['validation'].select(range(50))
#---

# train = train_dataset.select(range(1000))
# valid = valid_dataset.select(range(50))
# test = test_dataset.select(range(50))
#---
train = train_dataset
valid = valid_dataset
test = test_dataset

In [15]:
train[0]

{'sentence1': 'amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .',
 'sentence2': 'referring to him as only " the witness " , amrozi accused his brother of deliberately distorting his evidence .',
 'label': 1,
 'idx': 0}

In [16]:
from google.colab import userdata
HF_TOKEN= userdata.get('HuggingFace')

from huggingface_hub import login
login(token=HF_TOKEN)

In [41]:
# Load tokenizer
model_name = "answerdotai/ModernBERT-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True, padding="max_length", max_length=128)

In [42]:
tokenized_train = train.map(tokenize_function, batched=True)
tokenized_valid = valid.map(tokenize_function, batched=True)
tokenized_test = test.map(tokenize_function, batched=True)

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [43]:
tokenized_train

Dataset({
    features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'attention_mask'],
    num_rows: 3668
})

In [44]:
# Remove unnecessary columns and rename label column
tokenized_train = tokenized_train.remove_columns(["sentence1", "sentence2", "idx"])
tokenized_train = tokenized_train.rename_column("label", "labels")
tokenized_train.set_format("torch")
tokenized_train




Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 3668
})

In [45]:
tokenized_valid = tokenized_valid.remove_columns(["sentence1", "sentence2", "idx"])
tokenized_valid = tokenized_valid.rename_column("label", "labels")
tokenized_valid.set_format("torch")

#----------
tokenized_test = tokenized_test.remove_columns(["sentence1", "sentence2", "idx"])
tokenized_test = tokenized_test.rename_column("label", "labels")
tokenized_test.set_format("torch")
tokenized_test

Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 1725
})

In [46]:
tokenized_valid[0]

{'labels': tensor(1),
 'input_ids': tensor([50281,   248,   753,   253,  2739, 10613,  3376,  2136,  2506, 50276,
          1439,  4944,   253,  2567, 50276,   261,  1048,    14,  3945,  3116,
          5700,   964, 50282,     3,   253,  2739, 10613,  3376,  2136,  1057,
           417,  4944,   776,  1048,    14,  3945,  3116,  5700,   964, 50282,
         50283, 50283, 50283, 50283, 50283, 50283, 50283, 50283, 50283, 50283,
         50283, 50283, 50283, 50283, 50283, 50283, 50283, 50283, 50283, 50283,
         50283, 50283, 50283, 50283, 50283, 50283, 50283, 50283, 50283, 50283,
         50283, 50283, 50283, 50283, 50283, 50283, 50283, 50283, 50283, 50283,
         50283, 50283, 50283, 50283, 50283, 50283, 50283, 50283, 50283, 50283,
         50283, 50283, 50283, 50283, 50283, 50283, 50283, 50283, 50283, 50283,
         50283, 50283, 50283, 50283, 50283, 50283, 50283, 50283, 50283, 50283,
         50283, 50283, 50283, 50283, 50283, 50283, 50283, 50283, 50283, 50283,
         50283, 5

In [47]:
# Data collator for dynamic padding
#from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [48]:
#from transformers import TrainingArguments
# Define training arguments

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results_bert_model_1",         # Output directory for saved models
    learning_rate=2e-5,          # Learning rate
    per_device_train_batch_size=8, # Training batch size
    per_device_eval_batch_size=8, # Evaluation batch size
    num_train_epochs=10,            # Number of training epochs
    weight_decay=0.01,             # Weight decay
    #save_total_limit=2,            # Save only the 2 most recent models
    logging_dir="./logs",          # Log directory
    logging_steps=50,              # Log every 50 steps
    eval_strategy="epoch",   # Evaluate after each epoch
    save_strategy="epoch",         # Change save_strategy to 'epoch' to match eval_strategy
    load_best_model_at_end=True,    # Load the best model at the end of training
    metric_for_best_model="eval_loss",
    fp16=True,                       # Enabled mixed precision training
    gradient_accumulation_steps=2, # Added gradient accumulation
    #gradient_checkpointing=True,
    #max_grad_norm=1.0,  # Clip gradients with a maximum norm of 1.0
    report_to="none",
)

# training_args = TrainingArguments(
#     output_dir="./results_bert_model",         # Output directory for saved models
#     learning_rate=2e-5,          # Learning rate
#     per_device_train_batch_size=8, # Training batch size
#     per_device_eval_batch_size=8, # Evaluation batch size
#     num_train_epochs=10,            # Number of training epochs
#     weight_decay=0.01,             # Weight decay
#     #save_total_limit=2,            # Save only the 2 most recent models
#     logging_dir="./logs",          # Log directory
#     logging_steps=50,              # Log every 50 steps
#     eval_strategy="epoch",   # Evaluate after each epoch
#     save_strategy="epoch",         # Change save_strategy to 'epoch' to match eval_strategy
#     load_best_model_at_end=True,    # Load the best model at the end of training
#     metric_for_best_model="eval_loss",
#     fp16=True,                       # Enabled mixed precision training
#     gradient_accumulation_steps=2, # Added gradient accumulation
#     #gradient_checkpointing=True,
#     #max_grad_norm=1.0,  # Clip gradients with a maximum norm of 1.0
#     report_to="none",
# )


In [49]:
# Load Model
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2,  # Explicitly set for binary classification
)

Some weights of ModernBertForSequenceClassification were not initialized from the model checkpoint at answerdotai/ModernBERT-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [50]:
import os
# disable Weights and Biases
os.environ['WANDB_DISABLED']="true"


In [51]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Define a metric function
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)  # Get the class with the highest score

    # Calculate accuracy, precision, recall, and F1
    accuracy = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="binary")

    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1,
    }

In [52]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_valid, # Assuming you have a validation set
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    data_collator=data_collator,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],  # Early stopping
)

<ipython-input-52-128f9fc2c04a>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [29]:
#!nvidia-smi

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.908000,0.408431,0.816176,0.911290,0.810036,0.857685


In [31]:
valid_results = trainer.evaluate(tokenized_valid)
print("Evaluation Results:", valid_results)

Evaluation Results: {'eval_loss': 0.4065071940422058, 'eval_accuracy': 0.84, 'eval_precision': 0.8055555555555556, 'eval_recall': 0.9666666666666667, 'eval_f1': 0.8787878787878788, 'eval_runtime': 0.2873, 'eval_samples_per_second': 174.043, 'eval_steps_per_second': 24.366, 'epoch': 4.0}


In [32]:
# test_results = trainer.predict(tokenized_test_datasets)
# metrics = compute_metrics(test_results)
# print(metrics)

test_results = trainer.evaluate(tokenized_test)
print("Test Results:", test_results)

Test Results: {'eval_loss': 0.5479996204376221, 'eval_accuracy': 0.78, 'eval_precision': 0.8285714285714286, 'eval_recall': 0.8529411764705882, 'eval_f1': 0.8405797101449275, 'eval_runtime': 0.2582, 'eval_samples_per_second': 193.66, 'eval_steps_per_second': 27.112, 'epoch': 4.0}


In [33]:
# # Generate predictions for the test set
# predictions = test_results.predictions.argmax(-1)  # Predicted labels
# true_labels = test_results.label_ids               # True labels

# # Pair predictions with true labels
# for i in range(len(predictions)):
#     print(f"Sentence1: {tokenized_test_datasets[i]['sentence1']}")
#     print(f"Sentence2: {tokenized_test_datasets[i]['sentence2']}")
#     print(f"True Label: {true_labels[i]}, Predicted Label: {predictions[i]}\n")